In [1]:
import spiceypy as spice
import pvl
import os
import re
import subprocess
from ale import util
from itertools import chain
import io
import networkx as nx


# These should be provided when running this script. 
cube = "/home/tthatcher/Desktop/data/hybusa/hyb2.cub"
output_dir = "/home/tthatcher/Desktop/data/ale/hybusa/" # Output dir for created kernel files
data_dir = "/usgs/cpkgs/isis3/data/" # Dir of where to pull original kernels from
ckslicer_loc = "/home/tthatcher/Desktop/FORKS/ale/notebooks/ckslicer"

def merge_intervals(intervals):
    """
    Merge a set of intervals. The intervals are assumed to be closed, that is they include the end-points.
    
    Parameters
    ----------
    intervals : list
                The input list of intrvals where each interval is a tuple of (start, end)
    
    Returns
    -------
    : list
      A sorted list of the merged intervals
    """
    sorted_intervals = sorted(intervals, key=lambda tup: tup[0])
    merged = [sorted_intervals[0]]
    for interval in sorted_intervals[1:]:
        # No intersection
        if interval[0] > merged[-1][1]:
            merged.append(interval)
        # Intersection, but new interval isn't wholey contained
        elif interval[1] > merged[-1][1]:
            merged[-1] = (merged[-1][0], interval[1])
    return merged

def add_light_time_correction(cube_info, padding=5):
    """
    Compute the time intervals for the image and any light time correction
    
    Parameters
    ----------
    cube_info : ordered dict
                The cube info from ale.util.generate_kernels_from_cube
    padding : float
              Time padding in seconds to add to each interval
    
    Returns
    -------
    : list
      A sorted list of the intervals as (start_et, stop_et)
    """
    image_start_et = spice.scs2e(cube_info['SpacecraftID'], cube_info['SpacecraftClockCount'])
    image_end_et = image_start_et + cube_info['ExposureDuration'] * cube_info['Lines']
    
    inst_state, inst_lt = spice.spkez(cube_info['SpacecraftID'], image_start_et, 'J2000', 'LT+S', 0)
    target_state, target_lt = spice.spkez(cube_info['TargetID'], image_start_et, 'J2000', 'LT+S', 0)
    sun_state, sun_lt = spice.spkez(10, image_start_et, 'J2000', 'LT+S', cube_info['TargetID'])
    
    intervals = [
        (image_start_et - padding, image_end_et + padding),
        (image_start_et - padding - inst_lt, image_end_et + padding - inst_lt),
        (image_start_et - padding - target_lt, image_end_et + padding - target_lt),
        (image_start_et - padding - sun_lt, image_end_et + padding - sun_lt)]

    return merge_intervals(intervals)

/home/tthatcher/Desktop/FORKS/ale/ale/__init__.py:22: UserWarning: ALESPICEROOT environment variable not set, Spice Drivers will not function correctly
  warnings.warn('ALESPICEROOT environment variable not set, Spice Drivers will not function correctly')


In [7]:
pic = pvl.load(cube)
# spice.bods2c('HAYABUSA')
pic['IsisCube']['Kernels']

PVLGroup([
  ('NaifFrameCode', -37120)
  ('LeapSecond', '$base/kernels/lsk/naif0012.tls')
  ('TargetAttitudeShape', '$hayabusa2/kernels/pck/pck00010.tpc')
  ('TargetPosition', ['Table', '$hayabusa2/kernels/tspk/de430.bsp'])
  ('InstrumentPointing',
   ['Table',
    '$hayabusa2/kernels/ck/hyb2_hk_2015_v02.bc',
    '$hayabusa2/kernels/fk/hyb2_v10.tf'])
  ('Instrument', '$hayabusa2/kernels/ik/hyb2_onc_v00.ti')
  ('SpacecraftClock', '$hayabusa2/kernels/sclk/hyb2_20141203-20161231_v01.tsc')
  ('InstrumentPosition',
   ['Table',
    '$hayabusa2/kernels/spk/hyb2_20151123-20151213_0001m_final_ver1.oem.bsp'])
  ('InstrumentAddendum', '$hayabusa2/kernels/iak/hyb2oncAddendum0001.ti')
  ('ShapeModel', None)
  ('InstrumentPositionQuality', 'Reconstructed')
  ('InstrumentPointingQuality', 'Reconstructed')
  ('CameraVersion', 1)
])

In [3]:
# These are the processing steps. This will make use of the cube provided further up to create smaller, 
# more manageable kernel files for ale testing purposes. This currently only handles ck and spk files.

# Get dictionary of kernel lists from cube
cube_info = util.generate_kernels_from_cube(cube, format_as = 'dict')

# Replace path variables with absolute paths for kernels
for kernel_list in cube_info:
    for index, kern in enumerate(cube_info[kernel_list]):
        if kern is not None:
            cube_info[kernel_list][index] = data_dir + kern.strip('$')
            
# Create ordered list of kernels for furnishing
kernels = [kernel for kernel in chain.from_iterable(cube_info.values()) if isinstance(kernel, str)]
spice.furnsh(kernels)
                
# Loads cube as pvl to extract rest of data
cube_pvl = pvl.load(cube)
               
# Save other necesary info in cube_info dict
cube_info.update(Lines = cube_pvl['IsisCube']['Core']['Dimensions']['Lines'])
cube_info.update(Lines = 400)
cube_info.update(SpacecraftClockCount = cube_pvl['IsisCube']['Instrument']['SpacecraftClockStartCount'])
cube_info.update(ExposureDuration = cube_pvl['IsisCube']['Instrument']['ExposureDuration'].value * 0.001)
cube_info.update(TargetID = spice.bods2c(cube_pvl['IsisCube']['Instrument']['TargetName']))
cube_info.update(SpacecraftID = spice.bods2c(str(cube_pvl['IsisCube']['Instrument']['SpacecraftName'].replace('-',''))))

# Account for light time correction
intervals = add_light_time_correction(cube_info)

# For each binary ck kernel specified in cube, run the ckslicer, comment and to-transfer commands
for ck in [k for k in kernels if k.lower().endswith('.bc')]:
    ck_path, ck_file_extension = os.path.splitext(ck)
    ck_basename = os.path.basename(ck_path)
    for index, interval in enumerate(intervals):
        for frame in util.get_ck_frames(ck):
            output_basename = os.path.join(output_dir, ck_basename + '_' + str(index) + '_sliced_' + str(frame))
            output_kern = output_basename + ck_file_extension
            output_comments = output_basename + '.cmt'
            start_sclk = spice.sce2s(cube_info['SpacecraftID'], interval[0])
            end_sclk = spice.sce2s(cube_info['SpacecraftID'], interval[1])

            # Create new sliced ck kernel
            ckslicer_command = [ckslicer_loc, 
                                    '-LSK {}'.format(cube_info['LeapSecond'][0]), 
                                    '-SCLK {}'.format(cube_info['SpacecraftClock'][0]), 
                                    '-INPUTCK {}'.format(ck), 
                                    '-OUTPUTCK {}'.format(output_kern),
                                    '-ID {}'.format(frame),
                                    '-TIMETYPE {}'.format('SCLK'),
                                    '-START {}'.format(start_sclk),
                                    '-STOP {}'.format(end_sclk)]
            subprocess.run(ckslicer_command, check=True)

            # Remove old comments from new ck kernel
            commnt_command = ['commnt', '-d {}'.format(output_kern)]
            subprocess.run(commnt_command, check=True)

            with open(output_comments, 'w+') as comment_file:
                comment_file.write("This CK is for testing with the image: {}\n".format(cube))
                comment_file.write("\nThis CK was generated using the following command: {}\n")
                comment_file.write(" ".join(ckslicer_command))

            # Add new comments to new ck kernel
            new_commnts_command = ["commnt", "-a {}".format(output_kern), output_comments]
            subprocess.run(new_commnts_command, check=True)

            # Create the transfer file of the new ck kernel
            subprocess.run(["toxfr", output_kern], check=True)

# Create the config file for the spkmerge command
for index, interval in enumerate(intervals):
    output_spk_basename = os.path.join(output_dir, os.path.basename(os.path.splitext(cube)[0]) + '_' + str(index))
    output_spk = output_spk_basename + '.bsp'
    start_utc = spice.et2utc(interval[0], 'c', 3)
    end_utc = spice.et2utc(interval[1], 'c', 3)
    spk_dep_tree = util.create_spk_dependency_tree([k for k in kernels if k.lower().endswith('.bsp')])
    print(spk_dep_tree.nodes())
    config_string = util.spkmerge_config_string(spk_dep_tree,
                                                output_spk,
                                                [cube_info['TargetID'], cube_info['SpacecraftID'], 10],
                                                cube_info['LeapSecond'][0],
                                                start_utc,
                                                end_utc)
    with open(output_spk_basename + '.conf', 'w+') as spk_config:
        spk_config.write(config_string)

    # Create the new SPK
    spkmerge_command = ["spkmerge", spk_config.name]
    subprocess.run(spkmerge_command, check=True)

    # Create the transfer file of the new SPK kernel
    subprocess.run(["toxfr", output_spk], check=True)

/usgs/cpkgs/isis3/data/hayabusa2/kernels/tspk/de430.bsp
/usgs/cpkgs/isis3/data/hayabusa2/kernels/spk/hyb2_20151123-20151213_0001m_final_ver1.oem.bsp
[1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 199, 299, 301, 399, -37]
DEP:  [1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 199, 299, 301, 399, -37]
BOD:  399
DEP:  [1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 199, 299, 301, 399, -37]
BOD:  -37
DEP:  [1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 199, 299, 301, 399, -37]
BOD:  10
/usgs/cpkgs/isis3/data/hayabusa2/kernels/tspk/de430.bsp
/usgs/cpkgs/isis3/data/hayabusa2/kernels/spk/hyb2_20151123-20151213_0001m_final_ver1.oem.bsp
[1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 199, 299, 301, 399, -37]
DEP:  [1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 199, 299, 301, 399, -37]
BOD:  399
DEP:  [1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 199, 299, 301, 399, -37]
BOD:  -37
DEP:  [1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 199, 299, 301, 399, -37]
BOD:  10


In [4]:
# kernels = [k for k in kernels if k.lower().endswith('.bsp')]

# dep_tree = nx.DiGraph()

# for kernel in kernels:
#     brief = subprocess.run(["brief", "-c {}".format(kernel)],
#                            capture_output=True,
#                            check=True,
#                            text=True)
#     print(brief.stdout)
# #     results = re.findall(r'\((.*)\).*w\.r\.t\..*\((.*)\)', brief.stdout)
#     results = re.findall(r'([^()].\d+.*).w.r.t..+(..\d+)', brief.stdout)
# #     if not results:
# #         results = re.findall(r'(.\d+.*).w.r.t..+(.\d+)', brief.stdout)
    
#     for body, rel_body in results:
#         print(body.strip('( )'), rel_body.strip('( )'))
# #         dep_tree.add_edge(int(body), int(rel_body), kernel=kernel)
# #     x = re.findall(r'^.*\d+$', brief.stdout)
# #     print(x)

In [ ]:
import ale.drivers

with ale.drivers.hyb2_drivers.Hayabusa2IsisLabelNaifSpiceDriver